# Misure di complessità per Lorenz, Hénon e mappa logistica
Questo notebook genera tre serie temporali canoniche e applica le misure implementate in `measures.py`: autocorrelazione, ricostruzione di Takens, dimensione di correlazione, analisi di ricorrenza ed entropie.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from measures import (
    autocorrelation,
    integrated_autocorrelation_time,
    time_delay_embedding,
    correlation_dimension,
    recurrence_matrix,
    recurrence_quantification,
    permutation_entropy,
    sample_entropy,
    largest_lyapunov_rosenstein,
)
from timeseries_generators import lorenz63, henon_map, logistic_map


## Generazione delle serie
Simuliamo 8 000 passi scartando i primi 2 000 come transiente. Per il sistema di Lorenz analizziamo la coordinata `x`, per Hénon la componente `x` della coppia `(x, y)` e per la mappa logistica la serie stessa.

In [ ]:
lorenz_traj = lorenz63(rho=28.0, dt=0.01, steps=8000, transient=2000)
lorenz_series = lorenz_traj[:, 0]
lorenz_dt = 0.01

henon_traj = henon_map(a=1.4, b=0.3, steps=8000, transient=2000)
henon_series = henon_traj[:, 0]
henon_dt = 1.0

logistic_series = logistic_map(r=3.9, x0=0.5, steps=8000, transient=2000)
logistic_dt = 1.0

series_dict = {
    'Lorenz63': (lorenz_series, lorenz_dt),
    'Henon': (henon_series, henon_dt),
    'Logistic': (logistic_series, logistic_dt),
}


## Funzioni ausiliarie
Calcoliamo ritardi ottimali (primo valore in cui l'autocorrelazione scende sotto \(1/e\)), stimiamo la dimensione di correlazione con una regressione log-log e raccogliamo le misure richieste.

In [ ]:
def estimate_delay(acf):
    target = np.exp(-1)
    below = np.where(acf < target)[0]
    if below.size == 0:
        return 1
    lag = int(below[0])
    return max(1, lag)

def compute_measures(series, dt):
    acf = autocorrelation(series, max_lag=500)
    tau = estimate_delay(acf)
    emb_dim = 6
    embedded = time_delay_embedding(series, emb_dim, tau)
    radii = np.logspace(-3, 0, 24)
    _, corr_integral, corr_dim = correlation_dimension(
        series, emb_dim=emb_dim, delay=tau, radii=radii, max_points=4000, fit_range=(5, 18)
    )
    lyap = largest_lyapunov_rosenstein(series, dt, emb_dim=emb_dim, delay=tau)
    tau_int = integrated_autocorrelation_time(series, max_lag=500)
    subset = embedded[:2000]
    R, eps = recurrence_matrix(subset, percentage=0.05)
    rqa = recurrence_quantification(R)
    perm_ent = permutation_entropy(series, order=5, delay=1, normalize=True)
    samp_ent = sample_entropy(series, m=2, r=0.2)
    return {
        'tau_delay': tau,
        'tau_int': tau_int,
        'lyapunov': lyap,
        'corr_dim': corr_dim,
        'recurrence_eps': eps,
        'perm_entropy': perm_ent,
        'sample_entropy': samp_ent,
        **rqa,
    }


## Risultati numerici

In [ ]:
rows = []
for name, (series, dt) in series_dict.items():
    metrics = compute_measures(series, dt)
    metrics['serie'] = name
    rows.append(metrics)
results_df = pd.DataFrame(rows).set_index('serie')
results_df


## Visualizzazioni
Mostriamo la funzione di autocorrelazione e il recurrence plot per ciascuna serie.

In [ ]:
fig, axes = plt.subplots(len(series_dict), 2, figsize=(12, 4 * len(series_dict)))
for idx, (name, (series, dt)) in enumerate(series_dict.items()):
    acf = autocorrelation(series, max_lag=200)
    axes[idx, 0].plot(acf)
    axes[idx, 0].set_title(f'ACF - {name}')
    axes[idx, 0].set_xlabel('Lag')
    axes[idx, 0].set_ylabel('Autocorrelazione')
    tau = estimate_delay(acf)
    embedded = time_delay_embedding(series, 3, tau)[:500]
    R, eps = recurrence_matrix(embedded, percentage=0.05)
    axes[idx, 1].imshow(R, origin='lower', cmap='binary', interpolation='nearest')
    axes[idx, 1].set_title(f'Recurrence plot - {name} (ε={eps:.3f})')
    axes[idx, 1].set_xlabel('i')
    axes[idx, 1].set_ylabel('j')
plt.tight_layout()
plt.show()
